# #30DayMapChallenge
## Day 6: Dimensions

### Methodology References
1. https://www.acgeospatial.co.uk/time-series-on-landsat-data-gee/
2. https://medium.com/google-earth/visualizing-changing-landscapes-with-google-earth-engine-b2d502dc02a8

### Data Source
1. JRC Yearly Water Classification History, v1.4: https://developers.google.com/earth-engine/datasets/catalog/JRC_GSW1_4_YearlyHistory
2. Landsat 5/7/8/9

### Google Earth Engine Code

Download required images

### Create GIF
Put the required images in a folder and ensure they have a year in the filename

In [53]:
import os
import re
import numpy as np
from PIL import Image, ImageDraw, ImageFont
import rasterio
import matplotlib.pyplot as plt

In [25]:
# Function to create timelapse
def create_timelapse_gif(folder_path, output_gif, cmap_name='terrain', duration=500):
    # 🔤 Load font
    try:
        font = ImageFont.truetype("arial.ttf", 24)
    except IOError:
        font = ImageFont.load_default()

    # 🕒 Extract year from filename
    def extract_year(filename):
        match = re.search(r'\d{4}', filename)
        return match.group(0) if match else "Unknown"

    # 🎨 Apply color map to grayscale array
    def apply_colormap(array, cmap_name):
        cmap = plt.colormaps[cmap_name]  # Matplotlib ≥3.7
        norm = (array - array.min()) / (array.max() - array.min())
        colored = cmap(norm)
        rgb_array = (colored[:, :, :3] * 255).astype(np.uint8)
        return Image.fromarray(rgb_array)

    # 🖼️ Convert GeoTIFF to color image
    def geotiff_to_image(filepath, cmap_name):
        with rasterio.open(filepath) as src:
            array = src.read(1)
            array = np.nan_to_num(array)
            return apply_colormap(array, cmap_name)

    # 🧩 Process images
    frames = []
    for filename in sorted(os.listdir(folder_path)):
        if filename.lower().endswith('.tif'):
            year = extract_year(filename)
            filepath = os.path.join(folder_path, filename)
            img = geotiff_to_image(filepath, cmap_name)

            # ✏️ Add year label
            draw = ImageDraw.Draw(img)
            bbox = draw.textbbox((0, 0), year, font=font)
            text_width = bbox[2] - bbox[0]
            draw.text((img.width - text_width - 10, 10), year, font=font, fill="white")

            frames.append(img)

    # 🎞️ Save GIF
    if frames:
        frames[0].save(
            output_gif,
            save_all=True,
            append_images=frames[1:],
            duration=duration,
            loop=0
        )
        print(f"✅ GIF saved as {output_gif}")
    else:
        print("⚠️ No GeoTIFF images found in folder:", folder_path)

In [39]:
create_timelapse_gif(
    folder_path='./water',
    output_gif='jrc_water_dataset_timelapse.gif',
    cmap_name='terrain',
    duration=500  # milliseconds per frame
)

✅ GIF saved as jrc_water_dataset_timelapse.gif


In [37]:
create_timelapse_gif(
    folder_path='./landsat',
    output_gif='landsat_timelapse.gif',
    cmap_name='terrain',
    duration=500  # milliseconds per frame
)

✅ GIF saved as landsat_timelapse.gif


In [21]:
def create_natural_color_timelapse(folder_path, output_gif, duration=500):
    # 🔤 Load font
    try:
        font = ImageFont.truetype("arial.ttf", 24)
    except IOError:
        font = ImageFont.load_default()

    # 🕒 Extract year from filename
    def extract_year(filename):
        match = re.search(r'\d{4}', filename)
        return match.group(0) if match else "Unknown"

    # 🖼️ Convert GeoTIFF to natural color image (RGB composite)
    def geotiff_to_image(filepath):
        with rasterio.open(filepath) as src:
            # Read RGB bands (adjust if needed: 1=Red, 2=Green, 3=Blue)
            r = src.read(1).astype(float)
            g = src.read(2).astype(float)
            b = src.read(3).astype(float)

            # Normalize each band
            def normalize(array):
                array = np.nan_to_num(array)
                return ((array - array.min()) / (array.max() - array.min()) * 255).astype(np.uint8)

            rgb = np.stack([normalize(r), normalize(g), normalize(b)], axis=-1)
            return Image.fromarray(rgb)

    # 🧩 Process images
    frames = []
    for filename in sorted(os.listdir(folder_path)):
        if filename.lower().endswith('.tif'):
            year = extract_year(filename)
            filepath = os.path.join(folder_path, filename)
            img = geotiff_to_image(filepath)

            # ✏️ Add year label
            draw = ImageDraw.Draw(img)
            bbox = draw.textbbox((0, 0), year, font=font)
            text_width = bbox[2] - bbox[0]
            draw.text((img.width - text_width - 10, 10), year, font=font, fill="white")

            frames.append(img)

    # 🎞️ Save GIF
    if frames:
        frames[0].save(
            output_gif,
            save_all=True,
            append_images=frames[1:],
            duration=duration,
            loop=0
        )
        print(f"✅ GIF saved as {output_gif}")
    else:
        print(f"⚠️ No GeoTIFF images found in folder: {folder_path}")

In [33]:
create_natural_color_timelapse(
    folder_path='./landsat',
    output_gif='natural_color_landsat_timelapse.gif',
    duration=500  # milliseconds per frame
)

✅ GIF saved as natural_color_landsat_timelapse.gif
